In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from geo_dist_prep.data import GEONAMES_DB
from geo_dist_prep.schemas.training_data import TrainingData

# engine = create_engine(f"sqlite:///../{GEONAMES_DB}")
engine = create_engine(f"sqlite:///../.cache/geonames.db.britain.with-data")

Session = sessionmaker(bind=engine)
session = Session()

all_data_q = session.query(TrainingData)

all_data = pd.read_sql(all_data_q.statement, all_data_q.session.bind)
all_data.head()

all_data = all_data[all_data["distance"] < 50]
all_data = all_data[all_data["distance"] > 0]

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

shuff = all_data.sample(10000)

count = 0

# for _, row in shuff.iterrows():
#     for x, y in [(row['x1'], row['y1']), (row['x2'], row['y2'])]:
#         # if y < 0.574 or y > .578 or x < .798 or x > .802:
#         #     continue

#         count += 1

plt.hist(all_data["distance"], bins=1000)
# plt.scatter(shuff['x1'], shuff['y1'], s=0.5, color='red')
# plt.scatter(shuff['x2'], shuff['y2'], s=0.5, color='blue')
# y*180, x*360, s=2, color='red' if count % 2 == 0 else 'blue')
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU

from geo_dist_prep.model.visualize import plot_losses

from math import sqrt, pow


def distance(x1, y1, x2, y2):
    dist_y_sq = pow(y1 - y2, 2)
    dist_x_sq = pow(x1 - x2, 2)

    return sqrt(dist_x_sq + dist_y_sq)


from math import atan2, pi, cos, radians
from geo_dist_prep.normalize import normalize_coords


def get_data(lat1, lat2, lon1, lon2):
    y1, x1 = normalize_coords(row["lat1"], row["lon1"])
    y2, x2 = normalize_coords(row["lat2"], row["lon2"])
    pos = (x1 * y1 + x2 * y2) / 2

    delta_lat = row["lat2"] - row["lat1"]
    delta_lon = row["lon2"] - row["lon1"]

    angle = atan2(delta_lat, delta_lon) + pi
    direction = (angle % (2 * pi)) / (2 * pi)

    x1_km = row["lon1"] * 111.32 * cos(radians(row["lat1"]))
    x2_km = row["lon2"] * 111.32 * cos(radians(row["lat2"]))
    y1_km = row["lat1"] * 111.32
    y2_km = row["lat2"] * 111.32
    dist = distance(x1_km, y1_km, x2_km, y2_km)

    return {
        "pos": pos,
        "direction": direction,
        "sky_dist": dist,
    }


for row in all_data.iterrows():
    d = normalize(row[1])
    row[d.keys()] = d.values()

# all_data.apply(normalize, axis=1, result_type='expand')

# all_data["pos"] = (all_data["x1"] * all_data["y1"] + all_data["x2"] * all_data["y2"]) / 2
# all_data["sky_dist"] = all_data.apply(lambda row: distance(row['x1'], row['y1'], row['x2'], row['y2']), axis=1)
all_data["penalty"] = all_data["sky_dist"] / (all_data["distance"] / 40000)
# all_data["sky_dist"] = 1 - 1 / all_data["sky_dist"]
# all_data["dirmag"] = all_data["sky_dist"] * all_data["direction"]

features = [
    "pos",
    "direction",
    "sky_dist",
]
input_shape = len(features)
predictions = ["penalty"]
output_shape = len(predictions)

model = Sequential(
    [
        Dense(128, input_shape=(input_shape,)),
        Dense(32),
        BatchNormalization(),
        Dropout(0.2),
        Dense(32),
        LeakyReLU(),
        Dense(32),
        Dense(output_shape, activation="linear"),
    ]
)
model.compile(optimizer="adam", loss="mse")


# Split data into training and validation sets
data = all_data.copy()
# d2  = data.copy()
# d2.rename(columns={'x1': 'x2', 'x2': 'x1', 'y1': 'y2', 'y2': 'y1'}, inplace=True)
# data = data + d2

# scaler = StandardScaler()

# data = data.query(
#    '(0.574 <= lat1 <= 0.578) and (0.798 <= lon1 <= 0.802) and \
#    (0.574 <= lat2 <= 0.578) and (0.798 <= lon2 <= 0.802)'
# )

train = data.sample(frac=0.8, random_state=0)
val = data.drop(train.index)

# print(len(train))

# data = pd.DataFrame(scaler.fit_transform(data))

# Split data into features and labels
train_x = train[features]
train_y = train[predictions]
val_x = val[features]
val_y = val[predictions]

# Train model
model.fit(
    train_x,
    train_y,
    epochs=300,
    batch_size=5000,
    validation_data=(val_x, val_y),
    callbacks=[plot_losses],
)

# Evaluate model
model.evaluate(val_x, val_y)

In [ ]:
southport = 26700924
lat1, lon1 = -3.004175, 53.647599  # southport
lat2, lon2 = -2.9979, 50.7463  # lytham
# lat2, lon2 = -2.2522, 53.477 # manchester

In [ ]:
#
# Let's try to predict the density of a grid location
#
from geo_dist_prep.schemas.geoname import GeoName
from sqlalchemy import func, Integer, Float
import math


def lat_to_km(lat: float) -> float:
    """Converts degrees latitude to kilometers."""
    return lat * 111.32


def lon_to_km(lat: float, lon: float, lib=math) -> float:
    """Converts degrees longitude to kilometers at a given latitude."""
    return lon * 111.32 * lib.cos(lib.radians(lat))


def grid_coords(lat: float, lon: float, grid_size: float, lib=math) -> tuple[int, int]:
    """
    Returns the grid coordinate center for a given latitude and longitude and
    grid size in kilometers.
    """
    # latitude is always 111.32 km/degree
    y_offset_km = 90 * 111.32  # deal with negative latitudes
    y = lib.floor((lat_to_km(lat) + y_offset_km) / grid_size)

    x_km_degree = lon_to_km(lat, 1, lib=lib)  # km/degree at this latitude
    x_offset_km = 180 * x_km_degree  # deal with negative longitudes
    x = lib.floor((lon_to_km(lat, lon, lib=lib) + x_offset_km) / grid_size)

    y = (y * grid_size + grid_size / 2) / (111.32 * 180)
    x = (x * grid_size + grid_size / 2) / (111.32 * 360)

    return y, x


grid_data = pd.DataFrame()

for grid_size in range(10, 500, 10):
    print(grid_size)
    y, x = grid_coords(GeoName.lat, GeoName.lon, grid_size, lib=func)

    counter = func.count(GeoName.osm_id).label("count")
    grid = func.cast(grid_size, Integer).label("grid_size")
    density = func.cast(counter / grid_size ** 2, Float).label("density")
    table_plus_grid = (
        session.query(GeoName.osm_id)
        .add_columns(y.label("y"), x.label("x"))
        .add_columns(counter)
        .add_columns(density)
        .add_columns(func.sqrt(density).label("sqrt_density"))
        .add_columns(grid)
        .add_columns(func.cast(grid_size ** 2, Integer).label("sq_km"))
        .group_by(x, y, grid)
    )

    grid_data = pd.concat(
        [
            grid_data,
            pd.read_sql(table_plus_grid.statement, table_plus_grid.session.bind),
        ],
        ignore_index=True
    )

train = grid_data.sample(frac=0.8, random_state=0)
val = grid_data.drop(train.index)
test = grid_data.sample(frac=0.1, random_state=0)

grid_data


In [ ]:

# Split data into features and labels
features = [
    "x",
    "y",
    "sq_km",
]
input_shape = len(features)
predictions = ["sqrt_density"]
output_shape = len(predictions)

train_x = train[features]
train_y = train[predictions]
val_x = val[features]
val_y = val[predictions]

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
from keras.callbacks import Callback
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU


class PlotLosses(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_epoch_end(self, epoch, logs={}):
        loss_mse = logs.get("loss")
        val_loss_mse = logs.get("val_loss")
        self.losses.append(loss_mse)
        self.val_losses.append(val_loss_mse)

        clear_output(wait=True)
        plt.plot(self.losses[2:], label=f"loss ({self.losses[-1]:.4f})", alpha=0.5)
        plt.plot(self.val_losses[2:], label=f"val_loss ({self.val_losses[-1]:.4f})", alpha=0.5)
        plt.legend()
        plt.yscale("log")
        plt.show()


plot_losses = PlotLosses()

grid_model = Sequential(
    [
        Dense(256, input_shape=(input_shape,)),
        Dense(128),
        Dense(128),
        Dense(128),
        Dense(128),
        Dense(output_shape, activation="linear"),
    ]
)

def custom_loss(y_true, y_pred):
    #clipval = tf.clip_by_value(y_pred, 0.0, 1.0)
    penalty = 10
    loss = tf.square(y_true - y_pred)
    #loss += penalty * tf.where((y_pred < 0) | (y_pred > 1), tf.square(y_pred - 1), 0)

    # Adding distribution penalty
    distribution_penalty = tf.abs(1 / (y_pred / 500) - y_true)
    loss += 0.2 * distribution_penalty
    return tf.reduce_mean(loss)


grid_model.compile(optimizer="adam", loss='mse')


In [ ]:
import pandas as pd

# Train model
grid_model.fit(
    train_x,
    train_y,
    epochs=1000,
    batch_size=10_000,
    validation_data=(val_x, val_y),
    callbacks=[plot_losses],
)

df = pd.DataFrame(grid_model.predict(test[features]))

for i, prediction in enumerate(predictions):
    grid_data[prediction + "_pred"] = df[i]
    grid_data[prediction + "_pred_err"] = abs(df[i] - grid_data[prediction])


In [ ]:
plt.yscale('linear')
plt.hist(test["sqrt_density"], 50, alpha=0.5, label='sqrt_density')
#plt.hist([1/(x + 1) for x in range(len(test))], 50, alpha=0.5, label='myfunc')
plt.hist(grid_data["sqrt_density_pred"], 50, alpha=0.5, label='density_pred')
plt.legend()